<a href="https://colab.research.google.com/github/Ruheena-S/Hierarchical-classification-Loss-Functions-in-Image-Classification/blob/main/ResNet50_CIFAR100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip3 install torch_optimizer torchmetrics
# !nvidia-smi

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
#from torch_optimizer import Ranger
from torchvision.datasets import CIFAR100
from torch.utils.data import DataLoader
from torch.utils.data import random_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
torch.manual_seed(43)

# Set device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Define transforms for the dataset
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

In [ ]:
# Load the dataset

dataset = CIFAR100(root='./data', train=True, download=True, transform=transform_train)

test_dataset = CIFAR100(root='./data', train=False, download=True, transform=transform_test)


  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
val_size = 5000
train_size = len(dataset) - val_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])


trainloader = DataLoader(train_ds, batch_size=128, shuffle=True, num_workers=2, pin_memory=True)
valloader = DataLoader(val_ds, batch_size=100, num_workers=2, pin_memory=True)
testloader = DataLoader(test_dataset, batch_size =100, num_workers=4, pin_memory=True)

/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
# Define the ResNet50 model and freeze layers

model = torchvision.models.resnet50(pretrained=True)
model.conv1 = nn.Conv2d(3, 64, kernel_size = (3,3), padding = (1, 1), bias = False)
model.maxpool = nn.Identity()

for param in model.parameters():
    param.requires_grad = True
for param in model.layer4.parameters():
    param.requires_grad = True

model.fc = nn.Linear(2048, 100)

model.to(device)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): Identity()
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentu

In [ ]:
# Define the loss function and optimizer

criterion = nn.CrossEntropyLoss().to(device)
#optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)

# optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)
#optimizer = Ranger(model.parameters(), lr=0.001, weight_decay=0.005) 
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Train the model

BEST_VAL_METRIC = 0
BEST_MODEL = None


for epoch in range(20):  # number of epochs
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    for i, (inputs, labels) in enumerate(trainloader, 0):
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    print('Epoch: %d Train Loss: %.3f Train Acc: %.3f' % (epoch+1, train_loss/(i+1), 100.*correct/total))

    # Validate the model
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(valloader, 0):
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    print('Val Loss: %.3f Val Acc: %.3f' % (val_loss/(i+1), 100.*correct/total))

    if (100.*correct/total) > BEST_VAL_METRIC:
        BEST_VAL_METRIC = (100.*correct/total)
        BEST_MODEL = model.state_dict() 
        #torch.save(model, "/content/drive/MyDrive/MTP_Phase2/saved_models/resnet50_cifar100_OPT.pth")
        torch.save(model, "/content/drive/MyDrive/Ruheena/MTP/saved_models/resnet50_cifar100_OPT.pth")


Epoch: 1 Train Loss: 3.059 Train Acc: 23.978
Val Loss: 2.553 Val Acc: 32.300
Epoch: 2 Train Loss: 1.994 Train Acc: 44.884
Val Loss: 1.913 Val Acc: 46.820
Epoch: 3 Train Loss: 1.583 Train Acc: 54.524
Val Loss: 1.632 Val Acc: 53.860
Epoch: 4 Train Loss: 1.344 Train Acc: 60.596
Val Loss: 1.432 Val Acc: 58.700
Epoch: 5 Train Loss: 1.164 Train Acc: 65.607
Val Loss: 1.494 Val Acc: 57.820
Epoch: 6 Train Loss: 1.022 Train Acc: 69.233
Val Loss: 1.302 Val Acc: 63.840
Epoch: 7 Train Loss: 0.920 Train Acc: 72.364
Val Loss: 1.320 Val Acc: 61.900
Epoch: 8 Train Loss: 0.829 Train Acc: 74.531
Val Loss: 1.181 Val Acc: 66.520
Epoch: 9 Train Loss: 0.750 Train Acc: 76.909
Val Loss: 1.229 Val Acc: 65.680
Epoch: 10 Train Loss: 0.676 Train Acc: 79.016
Val Loss: 1.168 Val Acc: 67.440
Epoch: 11 Train Loss: 0.621 Train Acc: 80.578
Val Loss: 1.098 Val Acc: 69.520
Epoch: 12 Train Loss: 0.556 Train Acc: 82.289
Val Loss: 1.100 Val Acc: 69.060
Epoch: 13 Train Loss: 0.509 Train Acc: 83.620
Val Loss: 1.218 Val Acc: 67

KeyboardInterrupt: ignored

In [ ]:
# Test the model

model.load_state_dict(BEST_MODEL)

model.eval()
test_loss = 0.0
correct = 0
total = 0
with torch.no_grad():
    for i, (inputs, labels) in enumerate(testloader, 0):
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        test_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

print('Test Loss: %.3f Test Acc: %.3f' % (test_loss/(i+1), 100.*correct/total))


Test Loss: 1.271 Test Acc: 71.640
